# Retrieve Households

In [ ]:
from pathlib import Path
import zipfile
import io
import tempfile
import os

import geopandas as gpd
import shapely
import fiona
import numpy as np
import pandas as pd
import folium
import requests
import requests_cache

%matplotlib inline

In [ ]:
UKBUILDINGS_POINTS_FOLDER_PATH = Path('./data/ukbuildings/POINTS/')
UKBUILDINGS_POLYGONS_FOLDER_PATH = Path('./data/ukbuildings/POLYGONS/')
LONDON_BOUNDARY_FILE_URL = 'https://files.datapress.com/london/dataset/statistical-gis-boundary-files-london/2016-10-03T13:52:28/statistical-gis-boundaries-london.zip'
LONDON_HOUSING_SURVEY_URL = 'https://files.datapress.com/london/dataset/2011-census-housing/visualisation-data-housing.zip'

BOROUGH_SHAPE_FILE_PATH = Path('./statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp')
WARD_SHAPE_FILE_PATH = Path('./statistical-gis-boundaries-london/ESRI/London_Ward.shp')
HOUSING_FILE_PATH = Path('./HOUSING.xlsx')
BUILD_FOLDER = Path('./build')
BUILD_FOLDER.mkdir(parents=True, exist_ok=True)

In [ ]:
requests_cache.install_cache((BUILD_FOLDER / 'cache').as_posix())
RBCT_KML_FILE_PATH = Path((BUILD_FOLDER / 'rbct.kml').as_posix())

## Helper Functions

In [ ]:
HB_MIN_X = 500000
HB_MAX_X = 600000
HB_MIN_Y = 100000
HB_MAX_Y = 200000


def production_blocks(minx, miny, maxx, maxy):
    """Generator of GeoInformationGroup production blocks.
    
    Based on a rectangular bounding box defined in OS national grid, 
    this generator will yield all GeoInformationGroup production blocks 
    that are touched by the box.
    
    Supports only bounding boxes entirely in the HB production block
    reference.
    
    Parameters:
        * minx, miny, maxx, maxy: the parameters of the bounding box 
                                  defined in OS national grid
    
    Yields:
        The string name of each bounding box.
    """
    assert minx >= HB_MIN_X # supports only HB
    assert miny >= HB_MIN_Y # supports only HB
    assert maxx <= HB_MAX_X # supports only HB
    assert maxy <= HB_MAX_Y # supports only HB
    start_x = (int(minx) - HB_MIN_X) // 5000 + 1
    end_x = (int(maxx) - HB_MIN_X) // 5000 + 1
    start_y = (int(miny) - HB_MIN_Y) // 5000 + 1
    end_y = (int(maxy) - HB_MIN_Y) // 5000 + 1
    for x in range(start_x, end_x + 1):
        for y in range(start_y, end_y + 1):
            yield 'HB{:0>2}{:0>2}'.format(x, y)

assert set(production_blocks(500000, 100000, 500001, 100001)) == set(['HB0101'])
assert set(production_blocks(500000, 100000, 500000.1, 100000.1)) == set(['HB0101'])
assert set(production_blocks(505000, 100000, 505001, 100001)) == set(['HB0201'])
assert set(production_blocks(500000, 105000, 500001, 105001)) == set(['HB0102'])
assert set(production_blocks(500000, 100000, 505000, 100001)) == set(['HB0101', 'HB0201'])
assert set(production_blocks(500000, 100000, 500001, 105000)) == set(['HB0101', 'HB0102'])
assert set(production_blocks(504999, 100000, 505001, 100001)) == set(['HB0101', 'HB0201'])
assert set(production_blocks(504999.9, 100000, 505001, 100001)) == set(['HB0101', 'HB0201'])

In [ ]:
def ukbuildings_polygon_file(production_blocks):
    """Generator of file paths of UKBuilding production blocks.
    
    Parameters:
        * an iterable of production block names
        
    Yields:
        * file path of the file containing the production block
    """
    for production_block in production_blocks:
        yield list(UKBUILDINGS_POLYGONS_FOLDER_PATH.glob('{}*.shp'.format(production_block)))[0]

In [ ]:
_SOMEWHERE_IN_HARINGEY = [51.585978320592275, -0.00]


def aerial_base_map(location):
    """Creates an interactive aerial map at the given location.
    
    If there is no mapbox access token, an OpenStreetMap will be returned. 
    """
    try:
        mapbox_access_token = os.environ['MAPBOX_ACCESS_TOKEN']
    except KeyError:
        mapbox_access_token = None
    if mapbox_access_token is not None:
        base_map = folium.Map(
            location=location, 
            zoom_start=18, 
            tiles='https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v10/tiles/256/{{z}}/{{x}}/{{y}}?access_token={}'.format(mapbox_access_token),
            attr="© Mapbox, © OpenStreetMap",
            API_key='not necessary' # folium expects it here, but Mapbox expects it in the tiles url
        )
    else:
        base_map = default_base_map(location)
    return base_map


def default_base_map(location):
    """Creates an interactive OpenStreetMap at the given location."""
    return folium.Map(
            location=location,
            zoom_start=18,
            tiles='OpenStreetMap'
        )
            
def map_buildings(buildings, aerial=False):
    buildings = buildings.to_crs({'init': 'epsg:4258'})
    if aerial:
        base_map = aerial_base_map(_SOMEWHERE_IN_HARINGEY)
    else:
        base_map = default_base_map(_SOMEWHERE_IN_HARINGEY)
    bounds = [list(buildings.dissolve(by=lambda x: 1).geometry.iloc[0].envelope.boundary.coords)[i] for i in [0, 2]]
    bounds = [(lat, long) for long, lat in bounds]
    base_map.fit_bounds(bounds)
    with tempfile.TemporaryDirectory(prefix='geojson-') as geojson_dir:
        file_name = os.path.join(geojson_dir, 'temp.json')
        buildings.geometry.to_file(file_name, driver='GeoJSON')
        with open(file_name, 'r') as geojson:
            folium.GeoJson(geojson, name='geojson').add_to(base_map)
    return base_map

In [ ]:
def dissolve(df, by):
    """Dissolve retaining CRS.
    
    See https://github.com/geopandas/geopandas/pull/389.
    """
    crs = df.crs
    new_df = df.dissolve(by=by)
    new_df.crs = crs
    return new_df

## Read in Haringey Buildings

### Read in Haringey shape

In [ ]:
r = requests.get(LONDON_BOUNDARY_FILE_URL)
z = zipfile.ZipFile(io.BytesIO(r.content))
with tempfile.TemporaryDirectory(prefix='london-boundary-files') as tmpdir:
    z.extractall(path=tmpdir)
    borough_file = Path(tmpdir) / BOROUGH_SHAPE_FILE_PATH
    borough_data = gpd.read_file(borough_file.as_posix())
borough_data.plot()

In [ ]:
haringey = borough_data[borough_data.NAME == 'Haringey'].geometry.iloc[0]

In [ ]:
haringey.boundary

### Read all UKBuilding files that include Haringey buildings

Theoretically we could read all UKBuilding files, but the reading and especially the merging takes too long. So in a smarter way, let's filter all files not including Haringey buildings.

In [ ]:
raw_ukb_data = None
for shape_file_path in ukbuildings_polygon_file(production_blocks(*haringey.bounds)):
    print('Reading {}'.format(shape_file_path))
    shape_file_data = gpd.read_file(shape_file_path.as_posix())
    if raw_ukb_data is None:
        raw_ukb_data = shape_file_data
    else:
        raw_ukb_data = raw_ukb_data.append(shape_file_data)

In [ ]:
crs = raw_ukb_data.crs

In [ ]:
col_types = {
    'BASE': np.bool8,
    'BEC': np.int8,
    'BUNG': np.bool8,
    'DOR': np.int16,
    'DPS': np.int16,
    'GET': 'category',
    'MBN': 'category',
    'NAB': 'category',
    'RBCA': 'category',
    'RBCAT': 'category',
    'RBCC': 'category',
    'RBCS': 'category',
    'RBCT': 'category',
    'RBCTT': 'category',
    'RBN': np.int8,
    # TODO RBQ ??
    # TODO KBD ??
    'RDT': 'category',
    'RDTT': 'category',
    'RNR': 'category',
    'RRN': np.int8,
    'RRT': 'category',
    'RRTT': 'category',
    'RWN': np.int8,
    'RWT': 'category',
    'RWTT': 'category',
    'SBC': 'category'
}

In [ ]:
raw_ukb_data = raw_ukb_data.astype(col_types)
raw_ukb_data = gpd.GeoDataFrame(raw_ukb_data)

In [ ]:
raw_ukb_data.crs = crs

### Cut out Haringey

The read in files contain all buildings from all GeoInformationGroup production block files in which Haringey buildings are present. Let's filter for only Haringey buildings.

In [ ]:
from shapely.prepared import prep
haringey_prep = prep(haringey) # improves performace for the next step
in_haringey_mask = raw_ukb_data.geometry.map(haringey_prep.contains)

In [ ]:
raw_ukb_data = raw_ukb_data[in_haringey_mask]

### Tests

In [ ]:
ukb_poly = shapely.geometry.MultiPolygon([polygon for polygon in raw_ukb_data.geometry])

In [ ]:
assert ukb_poly.convex_hull.difference(haringey.convex_hull).area / 1000000 < 2
assert haringey.convex_hull.difference(ukb_poly.convex_hull).area / 1000000 < 2

The difference between the convex hull of all Haringey buildings in the UKBuildings dataset and the convex hull of the borough boundary is smaller than 2 * 2km<sup>2</sup>. _(Arbitrarily chosen to be small enough.)_

In [ ]:
len(raw_ukb_data)

## Reduce to Tottenham

As for the moment, let's look at Tottenham only.

In [ ]:
r = requests.get(LONDON_BOUNDARY_FILE_URL)
z = zipfile.ZipFile(io.BytesIO(r.content))
with tempfile.TemporaryDirectory(prefix='london-boundary-files') as tmpdir:
    z.extractall(path=tmpdir)
    ward_file = Path(tmpdir) / WARD_SHAPE_FILE_PATH
    ward_data = gpd.read_file(ward_file.as_posix())
ward_data.plot()

In [ ]:
tottenham = ward_data[(ward_data.BOROUGH == 'Haringey') & ward_data.NAME.map(lambda name: 'Tottenham' in name)]
tottenham = tottenham.dissolve(by=lambda x: 1).iloc[0].geometry
tottenham

In [ ]:
type(raw_ukb_data.dissolve(by='UBN'))

In [ ]:
tottenham_prep = prep(tottenham) # improves performace for the next step
in_tottenham_mask = raw_ukb_data.geometry.map(tottenham_prep.contains)
raw_ukb_data = raw_ukb_data[in_tottenham_mask]

map_buildings(dissolve(raw_ukb_data, by='UBN'), aerial=False)

In [ ]:
len(raw_ukb_data)

## Reference Number of Housholds

Retrieve the number of households in the area, based on the [2011 survey data](https://data.london.gov.uk/dataset/2011-census-housing).

In [ ]:
r = requests.get(LONDON_HOUSING_SURVEY_URL)
z = zipfile.ZipFile(io.BytesIO(r.content))
with tempfile.TemporaryDirectory(prefix='london-housing-files') as tmpdir:
    z.extractall(path=tmpdir)
    housing_file = Path(tmpdir) / HOUSING_FILE_PATH
    housing_data = pd.read_excel(
        housing_file, 
        sheetname='2011 Data',
        skiprows=[0],
        header=[0]
    )
housing_data.rename(columns={'Unnamed: 1': 'area_type'}, inplace=True)
housing_data['area_type'] = housing_data['area_type'].ffill()

In [ ]:
tottenham_housing_data = housing_data[(housing_data.DISTLABEL == 'Haringey') & 
                                      (housing_data.area_type == 'ward') & 
                                       housing_data.ZONELABEL.map(lambda label: 'Tottenham' in label)].sum()

In [ ]:
tottenham_housing_data.index

In [ ]:
cars_total = (
    tottenham_housing_data['No cars or vans in household'] +
    tottenham_housing_data['1 car or van in household'] +
    tottenham_housing_data['2 cars or vans in household'] +
    tottenham_housing_data['3 cars or vans in household'] +
    tottenham_housing_data['4 or more cars or vans in household']
)
ownership_total = (
    tottenham_housing_data['Owned: Total'] +
    tottenham_housing_data['Shared ownership (part owned and part rented)'] +
    tottenham_housing_data['Social rented: Total'] +
    tottenham_housing_data['Private rented: Total'] +
    tottenham_housing_data['Living rent free']
)
share_total = (
    tottenham_housing_data['Unshared dwelling: Total'] +
    tottenham_housing_data['Shared dwelling']
)
heating_total = (
    tottenham_housing_data['No central heating'] +
    tottenham_housing_data['Gas central heating'] +
    tottenham_housing_data['Electric (including storage heaters) central heating'] +
    tottenham_housing_data['Oil central heating'] +
    tottenham_housing_data['Solid fuel (for example wood, coal) central heating'] +
    tottenham_housing_data['Other central heating'] + 
    tottenham_housing_data['Two or more types of central heating']
    
)
assert cars_total == heating_total
assert cars_total == share_total
assert cars_total == ownership_total
cars_total

There are 11309 households in Tottenham.

## Dataset exploration

In [ ]:
raw_ukb_data.groupby('RDTT').UPN.count()

In [ ]:
raw_ukb_data.groupby('RDTT').UBN.apply(lambda values: len(set(values)))

In [ ]:
semi_detached_mask = (raw_ukb_data[raw_ukb_data.RDT == 3].groupby('UBN').UPN.count() > 2)

In [ ]:
semi_detached_mask[semi_detached_mask == True]

14 out of the 137 semi detached buildings have more than 2 distinct properties.

In [ ]:
detached_mask = (raw_ukb_data[raw_ukb_data.RDT == 4].groupby('UBN').UPN.count() > 1)

In [ ]:
detached_mask.describe()

In [ ]:
detached_mask.describe()

19 of 53 detached buildings have more than one property. They seem wrongly identified.

In [ ]:
print('Detached buildings with more than one property:')
map_buildings(raw_ukb_data[raw_ukb_data.UBN.map(lambda ubn: ubn in detached_mask[detached_mask == True].index)], aerial=True)

In [ ]:
def export_rbct_data_as_kml(df, path_to_file):
    """Exports RBCT data to KML to be used in Google Earth e.g."""
    fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
    export_data = df.copy()
    export_data.geometry = df.geometry.apply(lambda polygon: polygon.centroid)
    export_data = export_data[['RBCT', 'RBCTT', 'geometry']]
    export_data = export_data.astype({'RBCT': np.int8, 'RBCTT': np.str})
    export_data = gpd.GeoDataFrame(export_data)
    export_data.crs = df.crs
    export_data.to_file(path_to_file.as_posix(), driver='kml')

In [ ]:
export_rbct_data_as_kml(raw_ukb_data, RBCT_KML_FILE_PATH)

## From UKBuildings to Households

### Algorithm for inferring households

1. Group buildings by residential building type (RDT).
2. Assume 1 household per property in every house: Terraced (2), Semi-Detached (3), and Detached (4).
3. Assume several households for every flat type.
4. Assume households do not span upon storeys in flats.
5. Assume same household size for every household in a flat.
6. Find flat householf size for which the reference number of households from 2011 survey is reached (this will lead to households having different sizes).

In [ ]:
# TODO improve using Census 2011 data. The data has more details on how many flat households, 
# households in detached building etc

In [ ]:
ukb_data = raw_ukb_data.copy()

### Filter Non-Residential Buildings

In [ ]:
ukb_data.groupby(by='RNR').UPN.count()

In [ ]:
ukb_data = ukb_data[ukb_data.RNR != 2]

In [ ]:
len(ukb_data.UBN.unique())

In [ ]:
len(ukb_data.UPN.unique())

In [ ]:
11309 / 6679

There are 6679 residential properties in the UKBuildings data set, spread over 965 buildings. From the survey data we know that there are 11309 households in Tottenham, so we have in average 1.69 households per property.

### Handle NaN in RDT

In [ ]:
assert not any(ukb_data.RDT.isnull())

In [ ]:
ukb_data.RDT[ukb_data.RDT == 0].count()

No NaN's, but 262 values are 0 and hence missing.

In [ ]:
missing_value_mask = ukb_data.RDT == 0
ukb_data.RDT[missing_value_mask] = np.nan
ukb_data.RDTT[missing_value_mask] = np.nan

In [ ]:
assert (ukb_data.RDT.isnull()[ukb_data.RDT.isnull() == True]).count() == 262
assert (ukb_data.RDTT.isnull()[ukb_data.RDTT.isnull() == True]).count() == 262

In [ ]:
ukb_data.RDT[ukb_data.RDT == 0].count()

In [ ]:
ukb_data[missing_value_mask].RBCT.unique()

In [ ]:
ukb_data.RDT.fillna(method='ffill', inplace=True) # FIXME certainly not a good idea
ukb_data.RDTT.fillna(method='ffill', inplace=True) # FIXME certainly not a good idea

In [ ]:
assert not any(ukb_data.RDT.isnull())
assert not any(ukb_data.RDTT.isnull())

### Handle NaN in Height

In [ ]:
assert not any(ukb_data.HGT.isnull())

In [ ]:
ukb_data.HGT.min()

In [ ]:
missing_value_mask = ukb_data.HGT == 0
ukb_data.HGT[missing_value_mask] = np.nan

In [ ]:
average_building_height = ukb_data[~missing_value_mask].groupby('RDT').HGT.mean()
average_building_height

In [ ]:
for dwelling_type in ukb_data.RDT.unique():
    ukb_data.HGT[missing_value_mask & ukb_data.RDT == dwelling_type] = average_building_height[dwelling_type]

In [ ]:
assert not any(ukb_data.HGT.isnull())

In [ ]:
assert ukb_data.HGT.min() > 2, 'There is a building smaller than 2m.'

### Infer number of storeys

In [ ]:
# TODO UKBuilding is supposed to have that as a feature. Use it! (Current data set does not contain it though.)

In [ ]:
ASSUMED_STOREY_HEIGHT = 3.5

def simplistic_storey_number_detection(building_height):
    return round(building_height / ASSUMED_STOREY_HEIGHT)

assert simplistic_storey_number_detection(3.5) == 1
assert simplistic_storey_number_detection(7.0) == 2
assert simplistic_storey_number_detection(35) == 10
assert simplistic_storey_number_detection(3.6) == 1
assert simplistic_storey_number_detection(6.9) == 2
assert simplistic_storey_number_detection(3.5 + 3.5/2 - 0.001) == 1 
assert simplistic_storey_number_detection(3.5 + 3.5/2) == 2 

In [ ]:
ukb_data['number_storeys'] = ukb_data.HGT.map(simplistic_storey_number_detection) # FIXME No! Don't do that!

In [ ]:
ukb_data.number_storeys.describe()

### Determine number of households

In [ ]:
ukb_data['number_households'] = pd.Series(index=ukb_data.index, dtype=np.int8)

In [ ]:
ukb_data.ix[(ukb_data.RDT == 2) | (ukb_data.RDT == 3) | (ukb_data.RDT == 4), 'number_households'] = 1

In [ ]:
households_in_flats = 11309 - 5985
households_in_flats

There must be 5324 households in flats.

In [ ]:
# TODO handle mixed residential buildings. Their non-residential floor area should be subtracted.

def building_stock_difference(building_areas, building_storeys, target_household_number):
    assert len(building_areas) == len(building_storeys)
    def total_household_difference(target_household_size):
        total_number = 0
        for building_area, building_storey in zip(building_areas, building_storeys):
            total_number += number_households_in_building(building_area, building_storey, target_household_size[0])
        return abs(target_household_number - total_number)
    return total_household_difference


def number_households_in_building(building_area, building_storey, target_household_size):
    return round(building_area / target_household_size) * building_storey


assert number_households_in_building(10, 1, 10) == 1
assert number_households_in_building(10, 5, 10) == 5
assert number_households_in_building(20, 1, 10) == 2
assert number_households_in_building(14.9, 1, 10) == 1
assert number_households_in_building(15, 1, 10) == 2

In [ ]:
from scipy.optimize import fmin
optimal_flat_size = fmin(
    func=building_stock_difference(
        building_areas=[poly.area for poly in ukb_data[ukb_data.RDT == 1].geometry], 
        building_storeys=list(ukb_data[ukb_data.RDT == 1].number_storeys),
        target_household_number=households_in_flats
    ),
    x0=50
)[0]
optimal_flat_size

In [ ]:
def number_households(optimal_flat_size):
    def number_households(building):
        if building.RDT == 1:
            return number_households_in_building(building.geometry.area, building.number_storeys, optimal_flat_size)
        else:
            return building.number_households
    return number_households

In [ ]:
ukb_data['number_households'] = ukb_data.apply(
    number_households(optimal_flat_size),
    axis=1
)

In [ ]:
assert ukb_data.number_households.sum() == 11309

In [ ]:
ukb_data.number_households.describe()

In [ ]:
map_buildings(ukb_data[ukb_data.number_households > 100], aerial=False)